In [1]:
import pandas as pd
import pprint as pp

In [2]:
# Read Algo input csv
Algo_input = pd.read_csv("Algo_data_input.csv")
Algo_input

,living_wage,income,debt,time_to_repay,decision
0,33957,65383,57061,6.052419,Yes
1,24614,40961,50246,10.245713,No
2,29182,57060,129921,15.534472,No
3,34653,104308,53405,2.555691,Yes
4,30235,64336,100071,9.781825,Yes
5,24214,48841,56406,7.634710,Yes
6,32013,70617,54783,4.730339,Yes
7,32392,130567,73516,2.496087,Yes
8,25761,51612,48325,6.231223,Yes
9,29411,47528,13911,2.559475,Yes


In [3]:
target = Algo_input["decision"]
target_names = ["Yes", "No"]

In [4]:
data = Algo_input.drop(["decision","time_to_repay"], axis=1)
feature_names = data.columns
data.head()

,living_wage,income,debt
0,33957,65383,57061
1,24614,40961,50246
2,29182,57060,129921
3,34653,104308,53405
4,30235,64336,100071


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# Model Accuracy
# print('Test Acc: %.3f' % model.score(X_test, y_test))

# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print("SVM Model Classification Report")
print(classification_report(y_test, predictions,
                            target_names=target_names))

SVM Model Classification Report
              precision    recall  f1-score   support

         Yes       1.00      0.75      0.86         4
          No       0.90      1.00      0.95         9

    accuracy                           0.92        13
   macro avg       0.95      0.88      0.90        13
weighted avg       0.93      0.92      0.92        13



In [7]:
def Classify(State,Major_Category,Debt):
    import pymongo

    # Create connection variable
    conn = 'mongodb://localhost:27017'

    # Pass connection to the pymongo instance.
    dbconn = pymongo.MongoClient(conn)

    # Connect to a database. Will create one if not already available.
    db = dbconn.FortunEd

    Major_data = db.Majors.find_one({"Major_Category":Major_Category})

    Income_dict = {}
    for data in Major_data["Possible_Occupations"]:
        Occ = data["Occupation"]
        income_split =data["Median_Occ_Salary"].split(",")
        income = ''.join(map(str, income_split))
        low_25_income_split =data['Low_25_Occ_Salary'].split(",")
        low_25_pct_income = ''.join(map(str, low_25_income_split))
        recommended_ed = data['Recommended_Education']
        high_25_income_split = data['High_25_Occ_Salary'].split(",")
        high_25_pct_income =''.join(map(str, high_25_income_split)) 
        Income_dict[Occ] ={"Occ":Occ,
                          "median_income":pd.to_numeric(income,errors='coerce'),
                          "low_25_pct_income":pd.to_numeric(low_25_pct_income,errors='coerce'),
                          "high_25_pct_income":pd.to_numeric(high_25_pct_income,errors='coerce'),
                          "recommended_education":recommended_ed}
                           

    state_wages = db.StateWage

    state_wage_list = []
    state_wage_list = list(state_wages.find())
    result_values = [i[State] for i in state_wage_list if State in i]
    result =result_values[0]
    living_wage_split = result['living wage'].split("$")[1].split(",")
    wage_join =pd.to_numeric(''.join(map(str, living_wage_split)),errors='coerce')  



    output_df = pd.DataFrame(Income_dict.values())
    output_df.columns = ["Occupation","Median_Income","Low_25_pct_income","High_25_pct_income","Recommended_Education"]
    output_df["debt"] = Debt
    output_df["living_wage"] = wage_join
    output_df["major_category"] = Major_Category

    X = output_df.loc[:,["living_wage","Median_Income","debt"]]

    predictions = model.predict(X)
  
    output_df["decision"] = predictions
    outcome = output_df.to_dict("records")
    return output_df

Classify("Alaska","Arts",50000)

ValueError: invalid literal for int() with base 10: '*'